In [1]:
# Cell 1: Imports and constants
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from typing import Any
from product_vision_agent.agent import root_agent as product_vision_agent
from utils.schemes import InputSchema, OutputSchema

APP_NAME = "product_vision_app"
USER_ID = "alex"
SESSION_ID = "session1"

In [2]:
# Cell 2: async setup for session + runner
session_service_stateful = InMemorySessionService()

initial_state: dict[str, Any] = {
    "interaction_history": "",  # This will hold the concatenated context string
    "product_vision": "",
    "roadmap": "",
}


# NOTE: we're using `await` instead of asyncio.run(...)
session = await session_service_stateful.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state=initial_state,
)

runner = Runner(
    agent=product_vision_agent,
    app_name=APP_NAME,
    session_service=session_service_stateful,
)

In [3]:
requirements: str = "I want to open a store"

user_payload = InputSchema(
    unstructured_requirements=requirements,
)

content = types.Content(role="user", parts=[types.Part(text=str(user_payload))])

In [ ]:
import json
from typing import Any
from product_vision_agent.agent import InputSchema, OutputSchema
from google.genai import types


async def interact_with_agent() -> None:
    """Interactive refinement loop for the product vision agent."""

    while True:
        # --- Step 1: get user input ---
        requirements: str = input(
            "\nEnter your requirements for the product vision agent: "
        )

        # --- Step 2: wrap input according to schema ---
        user_payload = InputSchema(unstructured_requirements=requirements)
        content = types.Content(role="user", parts=[types.Part(text=str(user_payload))])

        # --- Step 3: run the agent ---
        async for event in runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content,
        ):

            if not event.is_final_response():
                continue

            # --- Step 4: process the final response ---
            if not (
                event.content and event.content.parts and event.content.parts[0].text
            ):
                print("⚠️  Final response: no valid text content found.")
                break

            final_text: str = event.content.parts[0].text
            try:
                raw_output: dict[str, Any] = json.loads(final_text)
                validated_output = OutputSchema(**raw_output)
            except Exception as exc:
                print(f"❌ Error parsing output: {exc}")
                break

            # --- Step 5: act on completeness flag ---
            if validated_output.is_complete:
                print("\n✅ Final product vision statement:")
                print(validated_output.product_vision_statement)
                return  # exit loop
            else:
                print("\nClarifying questions:")
                for question in validated_output.clarifying_questions:
                    print("-", question)

                print("\n" + "-" * 50)
                print("Current draft:")
                print(validated_output.product_vision_statement)
                print("-" * 50)

                print(
                    "Please provide additional requirements to refine the vision statement."
                )
                # The loop will prompt again automatically
                break  # re-enter outer while loop for the next round


# Run inside notebook: await interact_with_agent()

In [5]:
await interact_with_agent()

--- Session state updated with new history. ---

Clarifying questions:
- Who is the target user for this app?
- What specific problem or pain point does this app solve for the target user?
- What is the product category (e.g., mobile app, web app, SaaS) for this product?
- What is the primary benefit or outcome the user gains from the product?
- Who are the main competitors or alternative solutions the user might consider?
- What is the unique differentiator that sets this product apart from competitors?

✅ Final product vision statement:
For urban commuters and busy professionals who experience wasted time and stress from unpredictable public transport schedules, our product is a mobile app (smart transit assistant) that provides real-time route optimization and delay prediction to make commuting seamless. Unlike Google Maps, Citymapper, and Moovit, our product uses crowd-sourced motion data and AI forecasts to predict delays before official transport systems do.
--- Final vision save

In [ ]:
memory_state = await session_service_stateful.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

In [ ]:
type(memory_state)

In [ ]:
from google.adk.sessions import Session  # type hints only
from pprint import pprint

# Get the latest snapshot of the session from the service
current_session: Session = await session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

print("=== Session ID ===")
print(current_session.id)

print("\n=== Session state ===")
pprint(current_session.state)

print("\n=== Events so far ===")
for idx, event in enumerate(current_session.events):
    print(f"\n--- Event {idx} ---")
    print("type:", event.type if hasattr(event, "type") else "<no type>")
    # Many events carry LLM/user content in event.content.parts[*].text
    if getattr(event, "content", None):
        for part in event.content.parts:
            if getattr(part, "text", None):
                print("text:", part.text)

In [ ]:
# 1. Extract raw dict
raw_output = current_session.state["product_vision_assessment"]

# 2. Validate & parse using Pydantic
validated_output = OutputSchema(**raw_output)

# 3. Dump to JSON (validated)
json_str = validated_output.model_dump_json(indent=2)

print(json_str)

In [ ]:
current_session.state["product_vision_assessment"]

In [ ]:
user_input = input("Enter your requirements for the product vision agent: ")